# Actividad 03: Reglas de Asociación
> Estudiante: Mileydy Ninantay Diaz

> Código: 171915

---

## 1.- Instalar bibliotecas

In [12]:
import numpy as np  
import pandas as pd  
import itertools

# Otras librerías (para leer archivos con urls)
import requests
import io

In [13]:
#--configuración para mostrar el contenido completo de un dataframe
pd.set_option('display.max_colwidth', -1) 

C:\Users\USER\AppData\Local\Temp/ipykernel_10188/559992906.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## 2.- Funciones

### 2.1.- Funciones secundarias

**Función `mezclar_k_itemsets` :**
- INPUT: recibe un **diccionario** que contiene los **itemsets** y sus respectivos soportes **(support)**.
- OUTPUT: devuelve una **lista** de **itemsets**.
- Ésta función genera nuevos k-itemsets a partir de la combinación de k-1 itemsets.

In [14]:
def mezclar_k_itemsets(dicc):
  itemsets = [] #--lista para almacenar los itemsets generados
  keys = list(dicc.keys()) #--en la variable keys se almacena los itemsets del diccionario ingresado
  for i in range(len(keys)): #--recorre posiciones de la lista keys
    for j in range((i+1),len(keys)): #--recorre posiciones de la lista keys, a patir de la posición i+1
      if(len(keys[i]) == 1): #--para mezclar pares de 1-itemsets
          tupla = tuple(sorted((keys[i][0],keys[j][0]))) #--une dos 1-itemsets
          if(tupla not in itemsets):
            itemsets.append(tupla) #--si la tupla generada no esta en la lista itemsets, se agrega
      else: #--para mezclar pares de k-itemsets
        tupla_1 = tuple(sorted(keys[i])) #--se ordena la tupla lexicográficamente
        tupla_2 = tuple(sorted(keys[j])) #--se ordena la tupla lexicográficamente
        if (tupla_1 != tupla_2 and tupla_1[:len(tupla_1)-1] == tupla_2[:len(tupla_2)-1]):
          tupla = tuple(sorted(tupla_1 + (tupla_2[len(tupla_2)-1],))) #--si dos tuplas difieren solo en su último elemento, se mezcla ambas duplas
          if(tupla not in itemsets):
            itemsets.append(tupla) #--si la tupla generada no esta en la lista itemsets, se agrega
  return itemsets

### 2.2.- Funciones principales

**Función `get_frequent_itemsets` :**
- INPUT: recibe una **base de datos** en lista de listas que contiene las playlists, y el valor del soporte mínimo **(minsup)**.
- OUTPUT: devuelve un **diccionario** de los **itemsets frecuentes** y sus respectivos soportes **(support)**.
- Ésta función genera los **itemsets frecuentes** a partir de la base de  datos asignada, en este caso el conjunto de playlists. Además que cumplan que su soprte del itemset frecuentes sea mayor o igual que el soporte mínimo (minsup).

In [15]:
def get_frequent_itemsets(playlists, min_support):
  cant_playlists = len(playlists) #--en la variable se almacena la cantidad de filas de la base de datos
  '''Calcula el support de los 1-itemsets, luego filtra los 1-itemsets según el criterio minsup'''
  dic_support = {} #--se crea el diccionario para almacenar los 1-itemsets frecuentes con su support  
  data_extend = list(itertools.chain(*playlists)) #--convierte en una sola lista todas la claves del playlists
  for playlist in playlists: #--recorre cada fila de la base de datos (playlist)
    for song in playlist: #--recorre cada canción de la playlist
      if(song not in dic_support.keys()): 
        support = data_extend.count(song)/cant_playlists #--en caso que la canción aún no este en el diccionario se halla su support
        if(support >= min_support):
          dic_support[(song,)] = support #--si se cumple el criterio de minsup se agrega la cancion y su support
  dic_support = dict(sorted(dic_support.items())) #--ordena el diccionario lexicográficamente
  print(' -> Cantidad de 1-itemsets frecuentes: ', len(dic_support))
  
  '''Se crea los k-itemsets y se calcula sus supports, luego filtra los 1-itemsets según el criterio minsup'''
  itemsets = mezclar_k_itemsets(dic_support) #--a partir de los 1-itemsets frecuentes se genera los nuevos 2-itemsets 
  Dic_itemsets_freq = {} #--diccionario en el que se almacenarán los itemsets frecuentes y sus respectivos soportes
  cnt = 2 #--contador
  while(itemsets != []): #--se está en un bucle mientras se pueda generar todos los itemsets frecuentes
    dic_support.clear() #--se elimina todos los elementos del diccionario dic_support
    for itemset in itemsets: #--recorre cada tupla que hay en k_itemsets
      freq = 0 #--frecuencia inicializada en 0
      for playlist in playlists: #--recorre cada fila de la base de datos (playlist)
        if(set(itemset).issubset(set(playlist)) == True):
          freq += 1 #--si el itemset existe en un playlist de la base de datos, se va aumentando el valor de la frecuencia
      if (freq != 0):
        support = freq/cant_playlists #--si la frecuencia es diferente de 0, se halla el support
        if(support >= min_support):
          dic_support[itemset] = support #--si cumple con el criterio de minsup, se le halla su support de la tupla

    print(' -> Cantidad de',cnt,'itemsets frecuentes: ', len(dic_support))
    Dic_itemsets_freq.update(dic_support) #--se actualiza el diccionario que contiene los itemsets frecuentes       
    cnt += 1 #--contador se aumenta en 1
    itemsets = mezclar_k_itemsets(dic_support) #--a partir de los (k-1)-itemsets frecuentes se genera los nuevos k-itemsets
    
    if(itemsets == []):
      print('... No hay mas k-itemsets frecuentes.')
      break
  print('Cantidad total de k-itemsets frecuentes (longitud 2 a más): ', len(Dic_itemsets_freq))
  return Dic_itemsets_freq

**Función `generate_association_rules` :**
- INPUT: recibe una **base de datos** en lista de listas que contiene las playlists, un diccionario de los **itemsets frecuentes** y sus soportes, los umbrales mínimos de confianza **(minconf)** e interés **(lift)**.
- OUTPUT: devuelve un **diccionario** de las **reglas de asociacion** y sus respectivas **confianzas** y **lifts**.
- Ésta función genera las **reglas de asociacion** a partir de los itemsets frecuentes, cumpliendo con los criterios de **(minconf)** e interés **(lift)**.

In [16]:
def generate_association_rules(playlts, frequent_itemsets, confidence=0, lift=0):
  Diccionario = frequent_itemsets.copy() #--se copia el diccionario frequent_itemsets en otro diicionario
  Dic_Support = frequent_itemsets.copy() #--se copia el diccionario frequent_itemsets en el diicionario Dic_Support

  for key, value in  Diccionario.items(): #--recorre cada itesemt y su respectivo support
    Diccionario[key] = value*len(playlts) #--asigna al itemset su frecuencia (resulta de multiplicar su support con la cantidad de playlists de la BD)

  reglas = {} #--diccionario en el que se almacenarán las reglas de asociación
  for itemset in frequent_itemsets: #--recorre cada itesemt del frequent_itemsets
    item = tuple(sorted(itemset)) #--ordena lexicográficamente los elementos del itemset
    subconj = [] #--lista para almacenar los subconjuntos de un itemset
    for i in range(len(item)-1):
      subconj += itertools.combinations(item, i+1)  #--genera los subconjuntos (2 a más) del itemset

    reglas_x_item = [] #--almacena las posibles reglas generadas a parti de un itemset
    cont = 1 #--contador
    while (len(item) != cont): #--mientras el contador sea distinto al numero de subconjuntos del itemset
      antec = list( filter(lambda i: len(i) == cont, subconj)) #--lista los posibles antecedentes de una determinada cantidad de elementos
      conce = list( filter(lambda i: len(i) == len(item)-cont, subconj)) #--lista los posibles consecuentes de una determinada cantidad de elementos
      for i in antec: #--recorre los posibles antecedentes que formen una regla
        for j in conce: #--recorre los posibles consecuentes que formen una regla
          if (([i,j] not in reglas_x_item) and (len(set(i).intersection(set(j))) == 0)):
            #--si aun no existe la posible regla en reglas_x_item y si los elementos del antecedente y consecuente de la regla no se repiten, se agrega la regla
            reglas_x_item.append([i,j]) 
      cont += 1 #--aumenta el contador en uno

    for i in reglas_x_item: #--recorre cada regla almacenada en reglas_x_item
      numerador = i[0] + i[1]
      freq_num = 0 #--variable que almacena la frecuencia de la unión del antecedente y el consecuente de la regla
      freq_den = 0 #--variable que almacena la frecuencia del antecedente de la regla
      conf = 0 #--variable que almacena la confianza de una regla 

      '''halla la frecuencia del numerador (unión del antecedente y el consecuente de la regla)'''
      if (tuple(sorted(numerador)) in Diccionario.keys()): #--si la unión del antecedente y el consecuente de la regla está en el Diccionario
        freq_num = Diccionario.get(tuple(sorted(numerador))) #--devuelve la frecuencia de la unión del antecedente y el consecuente de la regla
      
      '''halla la frecuencia del denominador (antecedente de la regla)'''
      if (tuple(sorted(i[0])) in Diccionario.keys()): #--si el antecedente de la regla está en el Diccionario
        freq_den = (Diccionario.get(tuple(sorted(i[0])))) #--devuelve la frecuencia del antecedente de la regla
      else: #--en caso que no este agregado en el Diccionario
        for fila in playlts: #--recorre cada playlist de la BD
          if all(j in fila for j in tuple(i[0])):
            freq_den += 1 #--halla la frecuencia del antecedente de la regla
        Diccionario[tuple(sorted(i[0]))] = freq_den #--agrega el antecedente de la regla, y su frecuencia
        if(tuple(sorted(i[0])) in Dic_Support.keys()):
          Dic_Support[tuple(i[0])] = freq_den/len(playlts) #--si no esta en el diccionario Dic_Support, se agrega con su support

      if (freq_den == 0): #--si el valor es 0
        conf = 0 #--al valor de la confianza se le asigna 0
      else: #--caso contrario

        conf = freq_num/freq_den #--la confianza se halla dividiendo la frecuencia del numerador entre la frecuencia del antecedente de la regla
        #print('regla: ',i,conf)
        if(conf >= confidence): #--en caso que la confianza de la regla sea mayor o igual que el umbral mínimo de confianza
          #print('conf: ',i,conf)
          supp_num = 0 #--variable que almacena el support de la unión del antecedente y el consecuente de la regla
          supp_ant = 0 #--variable que almacena el support del antecedente de la regla
          supp_cons = 0 #--variable que almacena el support del consecuente de la regla
          lft = 0 #--variable que almacena el lift de la regla

          '''halla el support del numerador (unión del antecedente y el consecuente de la regla)'''
          if(tuple(sorted(numerador)) in Dic_Support.keys()): #--si la unión del antecedente y el consecuente de la regla está en el Dic_Support
            supp_num = Dic_Support.get(tuple(sorted(numerador))) #--devuelve el support de la unión del antecedente y el consecuente de la regla

          '''halla el support del antecedente de la regla'''
          if(tuple(sorted(i[0])) in Dic_Support.keys()): #--si el antecedente de la regla está en el Dic_Support
            supp_ant = Dic_Support.get(tuple(sorted(i[0]))) #--devuelve el support del antecedente de la regla
          else: #--en caso que no este agregado en el Dic_Support
            for fila in playlts: #--recorre cada playlist de la BD
              if all(k in fila for k in tuple(i[0])):
                supp_ant += 1 #--halla la frecuencia del antecedente de la regla
            supp_ant = supp_ant/len(playlts) #--determina el support
            Dic_Support[tuple(sorted(i[0]))] = supp_ant #--si no esta en el diccionario Dic_Support, se agrega con su support

          '''halla el support del consecuente de la regla'''
          if(tuple(sorted(i[1])) in Dic_Support.keys()): #--si el consecuente de la regla está en el Dic_Support
            supp_cons = Dic_Support.get(tuple(sorted(i[1]))) #--devuelve el support del consecuente de la regla
          else: #--en caso que no este agregado en el Dic_Support
            for fila in playlts: #--recorre cada playlist de la BD
              if all(k in fila for k in tuple(i[1])):
                supp_cons += 1 #--halla la frecuencia del consecuente de la regla
            supp_cons = supp_cons/len(playlts) #--determina el support
            Dic_Support[tuple(sorted(i[1]))] = supp_cons #--si no esta en el diccionario Dic_Support, se agrega con su support

          if (supp_ant == 0 or supp_cons == 0): #--si uno de los valores son 0
            lft = 0 #--al valor del lift se le asigna 0
          else: #--caso contrario
            lft = supp_num/(supp_ant*supp_cons) #--el lift se halla dividiendo la frecuencia del numerador entre la frecuencia del antecedente de la regla multiplicado por el consecuente de la regla
            if(lft >= lift): #--en caso que el lift de la regla sea mayor o igual que el criterio lift
              reglas[tuple(i)] = [supp_num, conf, lft] #--se agrega la regla en el diccionario Reglas junto a su confianza y lift           
  return reglas

## 3.- Importar datos 🟠

La **base de datos** a utilizar corresponde a **múltiples playlists** de la plataforma **spotify** creadas por usuarios de esta. Este es una muestra del dataset publicado para el Rec- Sys Challenge 2018.

In [17]:
#--Dirección del archivo en github
path = "https://raw.githubusercontent.com/mile1107/BIGDATA2021/main/ACTIVIDAD03/spotify.npy"

In [18]:
#--Obtiene el request de una url específica
response = requests.get(path)
response.raise_for_status()
#--Se carga la base de datos
data = np.load(io.BytesIO(response.content), allow_pickle=True)

## 4.- Preprocesamiento ⏮ 

- **Convertir a lista de listas la base de datos**

In [19]:
#--La base de datos cargada se almacenará en lista de listas, en la que cada lista será un conjunto de canciones o un playlist
play_lists = np.array([i for i in data.item().values()])

C:\Users\USER\AppData\Local\Temp/ipykernel_10188/558034997.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  play_lists = np.array([i for i in data.item().values()])


## 5.- Visualizar base de datos 

- **Cantidad de playlists de la base de datos**

In [20]:
print(len(play_lists))

10000


- **Muestra los 5 primeros playlists de la base de datos**

In [21]:
for i in range(5):
  print(i+1, ':', play_lists[i])
  print('')

1 : ['Lose Control (feat. Ciara & Fat Man Scoop)', 'Toxic', 'Crazy In Love', 'Rock Your Body', "It Wasn't Me", 'Yeah!', 'My Boo', 'Buttons', 'Say My Name', 'Hey Ya! - Radio Mix / Club Mix', 'Promiscuous', 'Right Where You Want Me - Radio Edit Version', 'Beautiful Soul', "Leavin'", 'Me & U', 'Ice Box', 'Sk8er Boi', 'Run It!', 'Check On It - feat. Bun B and Slim Thug', "Jumpin', Jumpin'", 'Soak Up The Sun', 'Where Is The Love?', "Stacy's Mom", 'Just The Girl', 'Yo (Excuse Me Miss)', 'Year 3000', 'Lip Gloss', 'Everytime We Touch - Radio Edit', 'Whatcha Say', 'Miss Independent', 'Party In The U.S.A.', 'The Great Escape', 'Replay', 'Forever', 'Your Love Is My Drug', 'Closer', 'One Less Lonely Girl', 'Paper Planes', 'Mr. Brightside', 'All The Small Things', 'Beep', 'Somebody To Love', 'Dirty Little Secret', 'Baby', 'A Thousand Miles', 'Livin on Sunday', 'See You Again', 'How Do You Sleep? - Featuring Ludacris', 'This Is Me', 'My Happy Ending', 'Check Yes Juliet', 'The Great Escape']

2 : ['F

- **Muestra los 5 últimos playlists de la base de datos**

In [22]:
for i in range(len(play_lists)-5,len(play_lists)):
  print(i+1, ':', play_lists[i])
  print('')

9996 : ['Scary Monsters And Nice Sprites', 'Bangarang (feat. Sirah)', 'Rap God', 'No Love', 'Blunt Blowin', 'Kings Never Die', 'Purple Lamborghini (with Rick Ross)', 'Wreak Havoc', 'For Those About to Rock (We Salute You)', 'Iron Man - Remastered Version', 'Wheel in the Sky', 'Separate Ways (Worlds Apart)', 'Wherever I May Roam', 'Headstrong', 'Paralyzer', 'In The End', 'Take Me Out', 'Requiem For A Tower', 'Edge of Seventeen - Remastered', 'Simple Man', 'Dust in the Wind', 'Paranoid - Remastered Version', "Beggin'", 'Beat It - Studio Version', 'All Of The Lights', 'Only Girl (In The World)', 'One', 'Sweet Dreams (Are Made of This) - Remastered', 'King For A Day', 'Hell Above', 'Bulls In The Bronx', 'Dive In', 'One Hundred Sleepless Nights', 'Stained Glass Eyes And Colorful Tears', 'Texas Is Forever', 'Mirror', "When I'm Gone", "X Gon' Give It To Ya", 'Radioactive', 'Faithfully', 'Bounce', 'The Way I Are', "Ain't Talkin' 'Bout Love - 2015 Remastered Version", 'Where Eagles Dare - 1998 

## 6.- Ejecutar algoritmo y mostrar resultados ⏮ 

### **Generación de itemsets frecuentes**

- Para la generación de itemsets frecuentes se escogió el valor de 0.015 para el minsup.
- Se ecogió este valor porque al probar con valores mayores a 0.0215 (como 0.025, 0.03 y 0.04) no se encuentran itemsets frecuentes de longitud mayor a 1. Mientras que al probar con el valor de 0.02 se encontró 3 itemsets frecuentes, por lo que considere que eran muy pocos, y así opté por el valor de 0.015.

In [23]:
itmsets_freq = get_frequent_itemsets(play_lists, 0.015)

 -> Cantidad de 1-itemsets frecuentes:  371
 -> Cantidad de 2 itemsets frecuentes:  22
 -> Cantidad de 3 itemsets frecuentes:  0
... No hay mas k-itemsets frecuentes.
Cantidad total de k-itemsets frecuentes (longitud 2 a más):  22


- **Muestra los itemsets frecuentes y su respectivo support**

In [27]:
df_itemsets = pd.DataFrame([[str(key[0]+' --> '+key[1]),itmsets_freq[key]] for key in itmsets_freq.keys()], columns=['Itemset frecuente', 'Support'])
df_itemsets

,Itemset frecuente,Support
0,Bad and Boujee (feat. Lil Uzi Vert) --> Bounce Back,0.0169
1,Bad and Boujee (feat. Lil Uzi Vert) --> Broccoli (feat. Lil Yachty),0.0155
2,Bad and Boujee (feat. Lil Uzi Vert) --> Caroline,0.0153
3,Bad and Boujee (feat. Lil Uzi Vert) --> HUMBLE.,0.0167
4,Bad and Boujee (feat. Lil Uzi Vert) --> Mask Off,0.0151
5,Bad and Boujee (feat. Lil Uzi Vert) --> iSpy (feat. Lil Yachty),0.0152
6,Bounce Back --> HUMBLE.,0.0156
7,Broccoli (feat. Lil Yachty) --> Caroline,0.0172
8,Broccoli (feat. Lil Yachty) --> No Problem (feat. Lil Wayne & 2 Chainz),0.0158
9,Closer --> Let Me Love You,0.0159


### **Generación de reglas de asociación**

A partir de los **22 itemsets frecuentes** obtenidos, se generaron todas las posibles reglas y se escogió solo a aquellas reglas de asociación que sobrepasan o son iguales a una **confianza del 51%** y el valor predeterminado de **1.0 de lift**. Además estas reglas filtradas son **las mejores 10 reglas de asociación**.

In [28]:
reglas_asociacion = generate_association_rules(play_lists,itmsets_freq, 0.51, 1.0)

- **Muestra las reglas de asociación y su respectivo support, confidence, lift**

In [30]:
#--Para una mejor visualización se convierte el diccionario a dataframe
df = []
for key in reglas_asociacion.keys():
    if(len(key[0]) == 1):
        if(len(key[1]) == 1):
            df.append([str(key[0][0]+' --> '+key[1][0]), reglas_asociacion[key][0], reglas_asociacion[key][1], reglas_asociacion[key][2]])
        else:
            df.append([str(key[0][0]+' --> '+key[1]), reglas_asociacion[key][0], reglas_asociacion[key][1], reglas_asociacion[key][2]])
    else:

        if(len(key[1]) == 1):
            df.append([str(key[0]+' --> '+key[1][0]), reglas_asociacion[key][0], reglas_asociacion[key][1], reglas_asociacion[key][2]])
        else:
            df.append([str(key[0]+' --> '+key[1]), reglas_asociacion[key][0], reglas_asociacion[key][1], reglas_asociacion[key][2]])

df_reglas = pd.DataFrame(df, columns=['Reglas de asociación', 'Support', 'Confidence', 'Lift'])
df_reglas

,Reglas de asociación,Support,Confidence,Lift
0,Bounce Back --> Bad and Boujee (feat. Lil Uzi Vert),0.0169,0.569024,16.493437
1,Bounce Back --> HUMBLE.,0.0156,0.525253,11.295753
2,Congratulations --> HUMBLE.,0.0214,0.531017,11.419728
3,Mask Off --> Congratulations,0.0162,0.512658,12.721048
4,XO TOUR Llif3 --> Congratulations,0.0179,0.554180,13.751354
5,DNA. --> HUMBLE.,0.0190,0.822511,17.688405
6,Mask Off --> HUMBLE.,0.0204,0.645570,13.883218
7,XO TOUR Llif3 --> HUMBLE.,0.0204,0.631579,13.582343
8,goosebumps --> HUMBLE.,0.0167,0.542208,11.660383
9,Mask Off --> XO TOUR Llif3,0.0163,0.515823,15.969746


## 7.- Resultado FINAL!!!   

En base a la información anterior, elegiremos las siguientes 4 reglas de asociación:

In [31]:
df_4_reglas = df_reglas.sort_values('Confidence',ascending=False)
df_4_reglas[0:4]

,Reglas de asociación,Support,Confidence,Lift
5,DNA. --> HUMBLE.,0.0190,0.822511,17.688405
6,Mask Off --> HUMBLE.,0.0204,0.645570,13.883218
7,XO TOUR Llif3 --> HUMBLE.,0.0204,0.631579,13.582343
0,Bounce Back --> Bad and Boujee (feat. Lil Uzi Vert),0.0169,0.569024,16.493437


- En la primera regla **'DNA. --> HUMBLE.'**, podemos ver que hay **190 playlists en la base de datos que contienen las dos canciones**. También podemos notar que mientras la canción 'DNA.' este en un playlist, hay un 82% de probabilidad de que el mismo playlist contenga la canción 'HUMBLE.', y además posee un lift de 17.69 que es un valor mayor a 1. 
**Información extra**: Las canciones 'DNA.' y 'HUMBLE.' pertenecen a un **mismo albúm de canciones llamado 'DAMN.'**, el cual es el cuarto álbum de estudio del **rapero estadounidense Kendrick Lamar**. Esto explica porque la **confianza es del 82%**, el **soporte de 0.019** y su **lift es el valor más alto de todas las reglas**.

- En la segunda regla **'Mask Off --> HUMBLE.'**, podemos ver que hay **240 playlists en la base de datos que contienen las dos canciones**. También podemos notar que mientras la canción 'Mask Off' este en un playlist, hay un 65% de probabilidad de que el mismo playlist contenga la canción 'HUMBLE.', y además posee un lift de 13.88 que es un valor mayor a 1. 
**Información extra**: La canción **'Mask Off'** es del **rapero Future**, el cual realizó un **remix** de la canción anteriormente mencionada, pero este remix se hizo con **colaboración** del **rapero Kendrick Lamar**, quién también canta la canción **'HUMBLE.'**. Esto explica porque la **confianza es del 65%** y hay 240 playlists que contienen ambas canciones.

- En la tercera regla **'XO TOUR Llif3 --> HUMBLE.'**, podemos ver que hay **240 playlists en la base de datos que contienen las dos canciones**. También podemos notar que mientras la canción 'XO TOUR Llif3' este en un playlist, hay un 63% de probabilidad de que el mismo playlist contenga la canción 'HUMBLE.', y además posee un lift de 13.58 que es un valor mayor a 1. 
**Información extra**: Las canciones **'XO TOUR Llif3'** y **'HUMBLE.'** estaba dentro del top 10 de las canciones más trasmitidas en el 2017, según el 'Top 10 of Everything 2019: The Ultimate Record Book of 2019'´, y ambas están en el 4 y 5 lugar respectivamente. Esto explica porque la **confianza es del 63%** y hay 240 playlists que contienen ambas canciones.

- En la cuarta regla **'Bounce Back --> Bad and Boujee (feat. Lil Uzi Vert)'**, podemos ver que hay **169 playlists en la base de datos que contienen las dos canciones**. También podemos notar que mientras la canción 'Bounce Back' este en un playlist, hay un 57% de probabilidad de que el mismo playlist contenga la canción 'Bad and Boujee (feat. Lil Uzi Vert)', y además posee un lift de 16.49 que es un valor mayor a 1. 
**Información extra**: La canción **'Bounce Back'** es del **rapero Big Sean**, y la canción **'Bad and Boujee (feat. Lil Uzi Vert)'** es de la **banda de trap Migos**, quién también canta la canción **'HUMBLE.'**, pero ambas canciones están dentro de un mashup (que significa la creación de una nueva canción a partir de la mezcla o pedazos de otras canciones) llamado **'Bad and Boujee x Bounce Back - Migos & Big Sean (Portada de William Singe)'**, este mashup la realizó William Singe, quién es un cantautor y compositor. Esto explica porque la **confianza es del 57%** y hay 169 playlists que contienen ambas canciones.

Se eligieron estas reglas ya que tenían un alto porcentaje de confianza y un alto soporte respecto a las otras reglas. Además, el lift de todas las reglas fue un número mayor a 1, lo que significa que hay más evidencias de que la regla no se debe a un artefacto aleatorio, es decir, mayor la evidencia de que la regla representa un patrón real.